In [1]:
import numpy as np
import torch
import random
seed=0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# load data

In [2]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [3]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\1.jpg,1
1,../dataset/wrist_xray/Normal\30.jpg,1
2,../dataset/wrist_xray/Fracture\47 Male (A View...,0
3,../dataset/wrist_xray/Normal\20.jpg,1
4,../dataset/wrist_xray/Normal\52.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [4]:
import sys
sys.path.append("../src") 

In [5]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [7]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [8]:
epoch=32
batchsize=8
lr=0.001
skip_tuning=True
aug=0

dim='gray' for gray scale images  
skip_tuning=False, no grid search applied for hyper-parameter tuning,  
aug=1-> pytorch augmentation  
aug=0-> no augmentation  
aug=2-> albumentation augmentation    
aug=3-> rand augmentation

# OurNet

In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.662   0.674     0.654
Class 1       0.576   0.548     0.540
macro         0.618   0.612     0.598
weighted      0.642   0.608     0.604
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.670   0.852     0.742
Class 1       0.678   0.418     0.498
macro         0.674   0.634     0.620
weighted      0.690   0.662     0.642
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.774   0.814     0.788
Class 1       0.724   0.676     0.688
macro         0.748   0.744     0.738
weighted      0.766   0.750     0.750
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.700   0.768     0.726
Class 1       0.640   0.566     0.588
macro         0.670   0.666     0.656
weighted      0.686   0.674     0.672
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.54,0.64,0.77,0.59
1,0.59,0.64,0.85,0.62
2,0.51,0.69,0.74,0.77
3,0.66,0.63,0.68,0.71
4,0.74,0.71,0.71,0.68
mean,0.608,0.662,0.75,0.674


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.590   0.614     0.594
Class 1       0.432   0.412     0.410
macro         0.512   0.514     0.504
weighted      0.534   0.522     0.522
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.620   0.892     0.724
Class 1       0.594   0.232     0.298
macro         0.606   0.564     0.510
weighted      0.624   0.610     0.548
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.656   0.682     0.662
Class 1       0.532   0.510     0.512
macro         0.594   0.598     0.588
weighted      0.616   0.604     0.606
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.632   0.690     0.660
Class 1       0.492   0.428     0.456
macro         0.562   0.560     0.556
weighted      0.584   0.592     0.584
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.72,0.74,0.74,0.72
1,0.56,0.62,0.56,0.54
2,0.46,0.59,0.59,0.62
3,0.37,0.55,0.47,0.50
4,0.50,0.55,0.66,0.58
mean,0.522,0.61,0.604,0.592


# ShuffleNet

In [11]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.584   0.562     0.570
Class 1       0.454   0.474     0.462
macro         0.520   0.518     0.518
weighted      0.542   0.528     0.532
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.652   0.804     0.700
Class 1       0.462   0.404     0.410
macro         0.556   0.604     0.554
weighted      0.576   0.608     0.568
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.608   0.586     0.578
Class 1       0.450   0.478     0.442
macro         0.524   0.530     0.510
weighted      0.548   0.526     0.518
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.626   0.768     0.682
Class 1       0.550   0.386     0.436
macro         0.588   0.578     0.556
weighted      0.606   0.594     0.578
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.44,0.49,0.44,0.56
1,0.69,0.59,0.67,0.64
2,0.54,0.64,0.49,0.69
3,0.68,0.74,0.61,0.55
4,0.29,0.58,0.42,0.53
mean,0.528,0.608,0.526,0.594


# Mnasnet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.546   0.580     0.558
Class 1       0.424   0.370     0.388
macro         0.486   0.474     0.474
weighted      0.508   0.504     0.498
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.620   0.924     0.732
Class 1       0.430   0.208     0.250
macro         0.526   0.566     0.492
weighted      0.540   0.610     0.522
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.616   0.608     0.606
Class 1       0.478   0.478     0.470
macro         0.548   0.544     0.538
weighted      0.570   0.560     0.554
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.608   0.664     0.632
Class 1       0.464   0.412     0.428
macro         0.536   0.536     0.530
weighted      0.558   0.566     0.554
--------------------------Accuracy Tab

d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\datasets\image dataset\siamese\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,DT,SVM,LR,KNN
0,0.54,0.49,0.59,0.41
1,0.64,0.64,0.51,0.54
2,0.49,0.69,0.62,0.67
3,0.53,0.68,0.58,0.58
4,0.32,0.55,0.50,0.63
mean,0.504,0.61,0.56,0.566


# MobileNet

In [13]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.614   0.620     0.614
Class 1       0.478   0.478     0.474
macro         0.546   0.550     0.546
weighted      0.566   0.560     0.560
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.602   0.868     0.702
Class 1       0.668   0.224     0.294
macro         0.636   0.550     0.496
weighted      0.656   0.580     0.522
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.596   0.630     0.602
Class 1       0.470   0.428     0.432
macro         0.532   0.528     0.518
weighted      0.554   0.538     0.536
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.660   0.632     0.612
Class 1       0.476   0.490     0.454
macro         0.568   0.560     0.530
weighted      0.594   0.560     0.544
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.54,0.51,0.51,0.59
1,0.41,0.56,0.54,0.59
2,0.64,0.62,0.51,0.59
3,0.68,0.55,0.55,0.50
4,0.53,0.66,0.58,0.53
mean,0.56,0.58,0.538,0.56


# DenseNet121

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=skip_tuning,aug=aug)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.702   0.640     0.658
Class 1       0.572   0.636     0.584
macro         0.634   0.638     0.620
weighted      0.658   0.632     0.630
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.712   0.796     0.750
Class 1       0.684   0.588     0.630
macro         0.698   0.692     0.688
weighted      0.710   0.704     0.700
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.744   0.698     0.710
Class 1       0.634   0.694     0.648
macro         0.690   0.696     0.680
weighted      0.710   0.684     0.686
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.682   0.694     0.686
Class 1       0.608   0.596     0.594
macro         0.646   0.648     0.640
weighted      0.662   0.650     0.650
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.59,0.62,0.54,0.46
1,0.69,0.77,0.79,0.74
2,0.67,0.82,0.67,0.69
3,0.63,0.68,0.76,0.68
4,0.58,0.63,0.66,0.68
mean,0.632,0.704,0.684,0.65
